In [19]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Layer, Dense, Dropout, Activation, Flatten, Reshape, Merge, Permute
from keras.layers import ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
import sys
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#from preprocessing.visualize_prepro import shiftedColorMap
import itertools
import tensorflow as tf

In [20]:
path = 'data/'
# input image dimensions
img_rows, img_cols = 320, 256

In [37]:
img_data_gen_args = dict(
        # featurewise_center=True,
        # featurewise_std_normalization=True,
        rescale=1. / 255,
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        fill_mode="constant",
        cval=0
    )

label_data_gen_args = dict(
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        fill_mode="constant",
        cval=1
    )

image_datagen = ImageDataGenerator(**img_data_gen_args)
mask_datagen = ImageDataGenerator(**label_data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
# image_datagen.fit(images, augment=True, seed=seed)
# mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    os.path.join(path, 'images/'),
    target_size=(img_rows, img_cols),
    class_mode=None,
    batch_size=4,
    shuffle=False,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    os.path.join(path, 'labels/'),
    target_size=(img_rows, img_cols),
    class_mode=None,
    batch_size=4,
    shuffle=False,
    color_mode='grayscale',
    seed=seed)


Found 80 images belonging to 1 classes.
Found 80 images belonging to 1 classes.


In [38]:
train_generator = zip(image_generator, mask_generator)

In [39]:
kernel = 3
filter_size = 64
pad = 1
pool_size = 2

In [40]:
model = Sequential()

model.add(Layer(input_shape=(img_rows, img_cols, 3)))
# encoding layers
model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(filter_size, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(128, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(256, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(512, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# decoding layers
model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(512, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(pool_size, pool_size)))
model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(256, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(pool_size, pool_size)))
model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(128, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())

model.add(UpSampling2D(size=(pool_size, pool_size)))
model.add(ZeroPadding2D(padding=(pad, pad)))
model.add(Conv2D(filter_size, (kernel,kernel), padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(1, (1,1), padding='valid'))
print(model.output_shape)
model.add(Reshape((img_rows * img_cols,)))
model.add(Activation('softmax'))

model.add(Reshape((img_rows, img_cols, 1)))
model.compile(loss="categorical_crossentropy", optimizer='adadelta',
              metrics=['accuracy'])

model.summary()

(None, 320, 256, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_9 (Layer)              (None, 320, 256, 3)       0         
_________________________________________________________________
zero_padding2d_65 (ZeroPaddi (None, 322, 258, 3)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 320, 256, 64)      1792      
_________________________________________________________________
batch_normalization_65 (Batc (None, 320, 256, 64)      256       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 160, 128, 64)      0         
_________________________________________________________________
zero_padding2d_66 (ZeroPaddi (None, 162, 130, 64)      0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 160, 128, 128)     7

In [49]:
from keras import models

def segnet(nClasses=11, input_height=256, input_width=320 ):

    kernel = 3
    filter_size = 64
    pad = 1
    pool_size = 2

    model = models.Sequential()
    model.add(Layer(input_shape=(3, input_height , input_width )))

    # encoder
    model.add(ZeroPadding2D(padding=(pad,pad)))
    model.add(Convolution2D(filter_size, kernel, kernel, border_mode='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size),dim_ordering="th"))

    model.add(ZeroPadding2D(padding=(pad,pad)))
    model.add(Convolution2D(128, kernel, kernel, border_mode='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size),dim_ordering="th"))

    model.add(ZeroPadding2D(padding=(pad,pad)))
    model.add(Convolution2D(256, kernel, kernel, border_mode='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(pool_size, pool_size),dim_ordering="th"))

    model.add(ZeroPadding2D(padding=(pad,pad)))
    model.add(Convolution2D(512, kernel, kernel, border_mode='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))


    # decoder
    model.add( ZeroPadding2D(padding=(pad,pad)))
    model.add( Convolution2D(512, kernel, kernel, border_mode='valid'))
    model.add( BatchNormalization())

    model.add( UpSampling2D(size=(pool_size,pool_size)))
    model.add( ZeroPadding2D(padding=(pad,pad)))
    model.add( Convolution2D(256, kernel, kernel, border_mode='valid'))
    model.add( BatchNormalization())

    model.add( UpSampling2D(size=(pool_size,pool_size)))
    model.add( ZeroPadding2D(padding=(pad,pad)))
    model.add( Convolution2D(128, kernel, kernel, border_mode='valid'))
    model.add( BatchNormalization())

    model.add( UpSampling2D(size=(pool_size,pool_size)))
    model.add( ZeroPadding2D(padding=(pad,pad)))
    model.add( Convolution2D(filter_size, kernel, kernel, border_mode='valid'))
    model.add( BatchNormalization())


    model.add(Convolution2D( nClasses , 1, 1, border_mode='valid',))

    model.outputHeight = model.output_shape[-2]

    model.outputWidth = model.output_shape[-1]

    model.add(Reshape(( nClasses ,  model.output_shape[-2]*model.output_shape[-1]   ), input_shape=( nClasses , model.output_shape[-2], model.output_shape[-1]  )))
    model.add(Permute((2, 1)))

    model.add(Activation('softmax'))
    model.compile(loss="categorical_crossentropy", optimizer='adadelta' , metrics=['accuracy'] )

In [50]:
checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=False)

In [51]:
model = segnet()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
  from ipykernel import kernelapp as app
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="valid")`
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="valid")`
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooli

ValueError: total size of new array must be unchanged

In [36]:
model.fit_generator(train_generator, samples_per_epoch=20, nb_epoch=5, callbacks=[checkpointer])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<zip objec..., callbacks=[<keras.ca..., steps_per_epoch=20, epochs=5)`
  """Entry point for launching an IPython kernel.


Epoch 1/5


ValueError: You are passing a target array of shape (4, 320, 256, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.